# Dataset Preparation

## <a id="contents"></a>Contents
- [Overview](#overview)
- [37NTF-A](#37ntf-a)
    - [Part 1](#37ntf-a--part-1)
    - [Part 2](#37ntf-a--part-2)
- [37NTF-B](#37ntf-b)
    - [Part 1](#37ntf-b--part-1)
    - [Part 2](#37ntf-b--part-2)
- [37NTF-C](#37ntf-c)
    - [Part 1](#37ntf-c--part-1)
    - [Part 2](#37ntf-c--part-2)

---

## <a id="overview"></a>Overview
- based on IEEE 37-Node Test Feeder
- downloaded from https://site.ieee.org/pes-testfeeders/resources/
- see sheet `IEEE 37-Node Test Feeder` in `./Ported Load Values.xlsx`
- Each phase of each load in the original system is treated as one consumer, provided that it has non-zero kilowatt (kW) value.
For example, the load at node 742 with 8 kW at phase A-B and 85 kW at phase B-C
is encoded as consumers with respective IDs 742-1 and 742-2 and respective loads 8 kW and 85 kW.
- total of 32 consumers
- only active power values are used
- To assign power rating for each appliance in consumer $c$, the interval $[0,L_{c}]$ is mapped unto the interval $[0,1.0]$,
wherein $N_{c} - 1$ points are sampled according to the uniform distribution $U [0.05,1.0)$.
The lengths of the resulting $N_{c}$ sub-intervals are fractions of $L_{c}$ that are the ratings of the $N_{c}$ appliances.
In principle, this means the appliance ratings add up to $L_{c}$.
In practice, appliance ratings are rounded to 5 decimal places,
and the deficit between $L_c$ and the sum of the appliance ratings is added to the rating of a randomly chosen appliance.
- each appliance is given priority level drawn from $U_{\text{d}} [1,P_{c}]$ where $P_{c}$ is the number of priority levels for consumer $c$
- three datasets:
    - 37NTF-A
    - 37NTF-B
    - 37NTF-C

---

## <a id="37ntf-a"></a>37NTF-A
- all consumers have $N_{c} = 100$ appliances
- all consumers have $P_{c} = 5$ priority levels
- saved in `./37NTF-A.xlsx`
- consumer IDs and their respective kW, no. of appliances, and no. of priority levels are in the sheet named `Overview`
- appliance ratings (in kW) and priority levels of each consumer are saved in a dedicated sheet named according to consumer IDs

### <a id="37ntf-a--part-1"></a>Part 1

In [1]:
import matplotlib, numpy, openpyxl, pandas

In [2]:
P_c = 5
M_c_per_consumer = 100

In [3]:
# Raw ported values
IEEE_37NTF_df = pandas.read_excel("./Ported Load Values.xlsx", sheet_name="IEEE 37-Node Test Feeder")

In [4]:
# Remove kVAr info
IEEE_37NTF_df = IEEE_37NTF_df.drop(columns=["kVAr"])

In [5]:
# Get consumer kW
L_c = IEEE_37NTF_df["kW"].to_numpy(dtype=numpy.float32)

In [6]:
# No. of appliances in each consumer
M_c = numpy.zeros(L_c.shape) + M_c_per_consumer

IEEE_37NTF_df["No. of appliances"] = M_c.astype(numpy.int64)

In [7]:
# Add no. of priority levels for each consumer
IEEE_37NTF_df["No. of priority levels"] = numpy.array([P_c for i in range(len(IEEE_37NTF_df["Consumer ID"]))])

In [8]:
IEEE_37NTF_df.head()

,Consumer ID,kW,No. of appliances,No. of priority levels
0,701-1,140,100,5
1,701-2,140,100,5
2,701-3,350,100,5
3,712-3,85,100,5
4,713-3,85,100,5


In [9]:
# For record keeping (08 October 2020)
workbook_path = "./37NTF-A.xlsx"
writer = pandas.ExcelWriter(workbook_path, engine="openpyxl")

IEEE_37NTF_df.to_excel(writer, sheet_name="Overview", index=False)
writer.save()
writer.close()

### <a id="37ntf-a--part-2"></a>Part 2

In [1]:
import matplotlib, numpy, openpyxl, pandas
from matplotlib import pyplot

In [2]:
workbook_path = "./37NTF-A.xlsx"
workbook = openpyxl.load_workbook(workbook_path)
writer = pandas.ExcelWriter(workbook_path, engine="openpyxl")
writer.book = workbook

IEEE_37NTF_df = pandas.read_excel("./37NTF-A.xlsx", sheet_name="Overview")
IEEE_37NTF_df.head()

,Consumer ID,kW,No. of appliances,No. of priority levels
0,701-1,140,100,5
1,701-2,140,100,5
2,701-3,350,100,5
3,712-3,85,100,5
4,713-3,85,100,5


In [3]:
# Partition consumer load into appliance ratings
for i in IEEE_37NTF_df.index:
    rand_points = numpy.random.uniform(0.05, 1.0, size=(IEEE_37NTF_df["No. of appliances"][i]-1,))
    # Just to make sure that no values are repeated
    while not (rand_points.shape[0] == numpy.unique(rand_points).shape[0]):
        rand_points = numpy.random.uniform(0.05, 1.0, size=(IEEE_37NTF_df["No. of appliances"][i]-1,))
    
    rand_points.sort()
    rand_points = numpy.append(numpy.array([0.0]), rand_points)
    rand_points = numpy.append(rand_points, numpy.array([1.0]))
    
    # Appliance ratings
    d_c = numpy.array([rand_points[i] - rand_points[i-1] for i in range(1,IEEE_37NTF_df["No. of appliances"][i]+1)])
    d_c = d_c * IEEE_37NTF_df["kW"][i]
    
    # Round appliance ratings to 5 decimal places
    d_c = numpy.around(d_c, decimals=5)
        
    # Just to make sure the sum of all appliance ratings equals the consumer rated load up to numerical precision
    deficit = float(IEEE_37NTF_df["kW"][i]) - numpy.sum(d_c)
    index_slack_appliance = numpy.random.randint(0, IEEE_37NTF_df["No. of appliances"][i])
    d_c[index_slack_appliance] += deficit
    
    # Assign priority levels to the appliances
    priority_levels = numpy.random.randint(1, IEEE_37NTF_df["No. of priority levels"][i]+1, size=d_c.shape)
    
    # Group appliances according to priority levels
    prioritized_d_c = []
    for p in range(1, IEEE_37NTF_df["No. of priority levels"][i]+1):
        priority_level_masks = priority_levels == p
        prioritized_d_c.append(d_c[priority_level_masks])
    
    # Construct dedicated DataFrame for a consumer
    consumer_df = pandas.concat([
        pandas.DataFrame({"Priority level "+str(p) : prioritized_d_c[p-1]}) \
        for p in range(1, IEEE_37NTF_df["No. of priority levels"][i]+1)
    ], axis=1)
    
    # Export consumer DataFrame as a dedicated sheet in spreadsheet
    consumer_df.to_excel(writer, sheet_name=IEEE_37NTF_df["Consumer ID"][i], index=False)

In [4]:
# For record keeping (08 October 2020)

writer.save()
writer.close()

---

## <a id="37ntf-b"></a>37NTF-B
- consumer with largest kW value $L_{\max}$ is given $N_{\max} = 100$ appliances
- any other consumer $c$ with kW value $L_{c}$ is given $N_{c} = \text{floor} \left(\frac{L_{c}}{L_{\max}} N_{\max}\right) + U_{\text{d}} [10,20]$,
where $\text{floor}$ is the floor function
and $U_{\text{d}} [10,20]$ is a discrete uniform distribution from the interval $[10,20]$
- all consumers have $P_{c} = 5$ priority levels
- saved in `./37NTF-B.xlsx`
- consumer IDs and their respective kW, no. of appliances, and no. of priority levels are in the sheet named `Overview`
- appliance ratings (in kW) and priority levels of each consumer are saved in a dedicated sheet named according to consumer IDs

### <a id="37ntf-b--part-1"></a>Part 1

In [1]:
import matplotlib, numpy, openpyxl, pandas

In [2]:
M_max = 100
P_c = 5

In [3]:
# Raw ported values
IEEE_37NTF_df = pandas.read_excel("./Ported Load Values.xlsx", sheet_name="IEEE 37-Node Test Feeder")

In [4]:
# Remove kVAr info
IEEE_37NTF_df = IEEE_37NTF_df.drop(columns=["kVAr"])

In [5]:
# Get consumer kW
L_c = IEEE_37NTF_df["kW"].to_numpy(dtype=numpy.float32)
L_max, index_L_max = L_c.max(), L_c.argmax()

In [6]:
# No. of appliances in each consumer
M_c = numpy.floor(M_max * L_c / L_max).astype(numpy.int64) + numpy.random.randint(10,20,L_c.shape)
M_c[index_L_max] = M_max

IEEE_37NTF_df["No. of appliances"] = M_c

In [7]:
# Add no. of priority levels for each consumer
IEEE_37NTF_df["No. of priority levels"] = numpy.array([P_c for i in range(len(IEEE_37NTF_df["Consumer ID"]))])

In [8]:
IEEE_37NTF_df.head()

,Consumer ID,kW,No. of appliances,No. of priority levels
0,701-1,140,53,5
1,701-2,140,58,5
2,701-3,350,100,5
3,712-3,85,42,5
4,713-3,85,39,5


In [9]:
# For record keeping (08 October 2020)
workbook_path = "./37NTF-B.xlsx"
writer = pandas.ExcelWriter(workbook_path, engine="openpyxl")

IEEE_37NTF_df.to_excel(writer, sheet_name="Overview", index=False)
writer.save()
writer.close()

### <a id="37ntf-b--part-2"></a>Part 2

In [1]:
import matplotlib, numpy, openpyxl, pandas
from matplotlib import pyplot

In [2]:
workbook_path = "./37NTF-B.xlsx"
workbook = openpyxl.load_workbook(workbook_path)
writer = pandas.ExcelWriter(workbook_path, engine="openpyxl")
writer.book = workbook

IEEE_37NTF_df = pandas.read_excel("./37NTF-B.xlsx", sheet_name="Overview")
IEEE_37NTF_df.head()

,Consumer ID,kW,No. of appliances,No. of priority levels
0,701-1,140,53,5
1,701-2,140,58,5
2,701-3,350,100,5
3,712-3,85,42,5
4,713-3,85,39,5


In [3]:
# Partition consumer load into appliance ratings
for i in IEEE_37NTF_df.index:
    rand_points = numpy.random.uniform(0.05, 1.0, size=(IEEE_37NTF_df["No. of appliances"][i]-1,))
    # Just to make sure that no values are repeated
    while not (rand_points.shape[0] == numpy.unique(rand_points).shape[0]):
        rand_points = numpy.random.uniform(0.05, 1.0, size=(IEEE_37NTF_df["No. of appliances"][i]-1,))
    
    rand_points.sort()
    rand_points = numpy.append(numpy.array([0.0]), rand_points)
    rand_points = numpy.append(rand_points, numpy.array([1.0]))
    
    # Appliance ratings
    d_c = numpy.array([rand_points[i] - rand_points[i-1] for i in range(1,IEEE_37NTF_df["No. of appliances"][i]+1)])
    d_c = d_c * IEEE_37NTF_df["kW"][i]
    
    # Round appliance ratings to 5 decimal places
    d_c = numpy.around(d_c, decimals=5)
        
    # Just to make sure the sum of all appliance ratings equals the consumer rated load up to numerical precision
    deficit = float(IEEE_37NTF_df["kW"][i]) - numpy.sum(d_c)
    index_slack_appliance = numpy.random.randint(0, IEEE_37NTF_df["No. of appliances"][i])
    d_c[index_slack_appliance] += deficit
    
    # Assign priority levels to the appliances
    priority_levels = numpy.random.randint(1, IEEE_37NTF_df["No. of priority levels"][i]+1, size=d_c.shape)
    
    # Group appliances according to priority levels
    prioritized_d_c = []
    for p in range(1, IEEE_37NTF_df["No. of priority levels"][i]+1):
        priority_level_masks = priority_levels == p
        prioritized_d_c.append(d_c[priority_level_masks])
    
    # Construct dedicated DataFrame for a consumer
    consumer_df = pandas.concat([
        pandas.DataFrame({"Priority level "+str(p) : prioritized_d_c[p-1]}) \
        for p in range(1, IEEE_37NTF_df["No. of priority levels"][i]+1)
    ], axis=1)
    
    # Export consumer DataFrame as a dedicated sheet in spreadsheet
    consumer_df.to_excel(writer, sheet_name=IEEE_37NTF_df["Consumer ID"][i], index=False)

In [4]:
# For record keeping (08 October 2020)

writer.save()
writer.close()

---

## <a id="37ntf-c"></a>37NTF-C
- all consumers have $N_{c} = 100$ appliances
- a consumer has $P_{c} = U_{\text{d}} [1,5]$ priority levels
- saved in `./37NTF-C.xlsx`
- consumer IDs and their respective kW, no. of appliances, and no. of priority levels are in the sheet named `Overview`
- appliance ratings (in kW) and priority levels of each consumer are saved in a dedicated sheet named according to consumer IDs

### <a id="37ntf-c--part-1"></a>Part 1

In [1]:
import matplotlib, numpy, openpyxl, pandas

In [2]:
M_c_per_consumer = 100

In [3]:
# Raw ported values
IEEE_37NTF_df = pandas.read_excel("./Ported Load Values.xlsx", sheet_name="IEEE 37-Node Test Feeder")

In [4]:
# Remove kVAr info
IEEE_37NTF_df = IEEE_37NTF_df.drop(columns=["kVAr"])

In [5]:
# Get consumer kW
L_c = IEEE_37NTF_df["kW"].to_numpy(dtype=numpy.float32)

In [6]:
# No. of appliances in each consumer
M_c = numpy.zeros(L_c.shape) + M_c_per_consumer

IEEE_37NTF_df["No. of appliances"] = M_c.astype(numpy.int64)

In [7]:
# Add no. of priority levels for each consumer
IEEE_37NTF_df["No. of priority levels"] = numpy.random.randint(1,6,(len(IEEE_37NTF_df["Consumer ID"]),))

In [8]:
IEEE_37NTF_df.head()

,Consumer ID,kW,No. of appliances,No. of priority levels
0,701-1,140,100,2
1,701-2,140,100,2
2,701-3,350,100,5
3,712-3,85,100,2
4,713-3,85,100,1


In [9]:
# For record keeping (11 October 2020)
workbook_path = "./37NTF-C.xlsx"
writer = pandas.ExcelWriter(workbook_path, engine="openpyxl")

IEEE_37NTF_df.to_excel(writer, sheet_name="Overview", index=False)
writer.save()
writer.close()

### <a id="37ntf-c--part-2"></a>Part 2

In [1]:
import matplotlib, numpy, openpyxl, pandas
from matplotlib import pyplot

In [2]:
workbook_path = "./37NTF-C.xlsx"
workbook = openpyxl.load_workbook(workbook_path)
writer = pandas.ExcelWriter(workbook_path, engine="openpyxl")
writer.book = workbook

IEEE_37NTF_df = pandas.read_excel("./37NTF-C.xlsx", sheet_name="Overview")
IEEE_37NTF_df.head()

,Consumer ID,kW,No. of appliances,No. of priority levels
0,701-1,140,100,2
1,701-2,140,100,2
2,701-3,350,100,5
3,712-3,85,100,2
4,713-3,85,100,1


In [3]:
# Partition consumer load into appliance ratings
for i in IEEE_37NTF_df.index:
    rand_points = numpy.random.uniform(0.05, 1.0, size=(IEEE_37NTF_df["No. of appliances"][i]-1,))
    # Just to make sure that no values are repeated
    while not (rand_points.shape[0] == numpy.unique(rand_points).shape[0]):
        rand_points = numpy.random.uniform(0.05, 1.0, size=(IEEE_37NTF_df["No. of appliances"][i]-1,))
    
    rand_points.sort()
    rand_points = numpy.append(numpy.array([0.0]), rand_points)
    rand_points = numpy.append(rand_points, numpy.array([1.0]))
    
    # Appliance ratings
    d_c = numpy.array([rand_points[i] - rand_points[i-1] for i in range(1,IEEE_37NTF_df["No. of appliances"][i]+1)])
    d_c = d_c * IEEE_37NTF_df["kW"][i]
    
    # Round appliance ratings to 5 decimal places
    d_c = numpy.around(d_c, decimals=5)
        
    # Just to make sure the sum of all appliance ratings equals the consumer rated load up to numerical precision
    deficit = float(IEEE_37NTF_df["kW"][i]) - numpy.sum(d_c)
    index_slack_appliance = numpy.random.randint(0, IEEE_37NTF_df["No. of appliances"][i])
    d_c[index_slack_appliance] += deficit
    
    # Assign priority levels to the appliances
    priority_levels = numpy.random.randint(1, IEEE_37NTF_df["No. of priority levels"][i]+1, size=d_c.shape)
    
    # Group appliances according to priority levels
    prioritized_d_c = []
    for p in range(1, IEEE_37NTF_df["No. of priority levels"][i]+1):
        priority_level_masks = priority_levels == p
        prioritized_d_c.append(d_c[priority_level_masks])
    
    # Construct dedicated DataFrame for a consumer
    consumer_df = pandas.concat([
        pandas.DataFrame({"Priority level "+str(p) : prioritized_d_c[p-1]}) \
        for p in range(1, IEEE_37NTF_df["No. of priority levels"][i]+1)
    ], axis=1)
    
    # Export consumer DataFrame as a dedicated sheet in spreadsheet
    consumer_df.to_excel(writer, sheet_name=IEEE_37NTF_df["Consumer ID"][i], index=False)

In [4]:
# For record keeping (11 October 2020)

writer.save()
writer.close()

---